In [ ]:
import pandas as pd
import numpy as np
import random 
# Read in the data
df = pd.read_csv('movies_metadata.csv')
# select the rows between 50 and 1000   
df = df.iloc[0:80000]

# Create a new dataset with the columns: genres, id, title, release_date, revenue, vote_average
df = df[['id', 'genres', 'title', 'overview', 'release_date', 'revenue', 'vote_average', 'vote_count']]
# if id cannot be converted to int, remove the row from the dataframe
df = df[pd.to_numeric(df['id'], errors='coerce').notnull()]
df['id'] = df['id'].astype(int)
df.drop_duplicates(subset=['id'], inplace=True)
df = df.reset_index(drop=True)
# turn genres field from array of ids and names into a list of names
df['genres'] = df['genres'].apply(lambda x: eval(x) if isinstance(x, str) else np.nan)
df['genres'] = df['genres'].apply(lambda x: [i['name'] for i in x] if isinstance(x, list) else np.nan)
df.head()

# Create new dataframe with just id and genres
df_genres = df[['id', 'genres']]

# Create a new row for each genre in the dataframe
df_genres = df_genres.explode('genres')

# Remove genre from df
df = df.drop(columns=['genres'])

# Change the column name genres to genre
df_genres = df_genres.rename(columns={'genres': 'genre'})

# Find ids in df_genres that are not in df
df_genres = df_genres.reset_index(drop=True)
df.drop_duplicates(subset=['id'], inplace=True)
df = df.reset_index(drop=True)
# drop rows if any of its column is null
df = df.dropna()
df_genres = df_genres.dropna()

In [ ]:
df_credits = pd.read_csv('credits.csv')
df_credits = df_credits[df_credits['id'].isin(df['id'])]
df_credits = df_credits.reset_index(drop=True)

In [ ]:
#sort by id
df = df.sort_values(by=['id'])

In [ ]:
df_credits['cast'] = df_credits['cast'].apply(lambda x: eval(x) if isinstance(x, str) else np.nan)

data = []
for i in df_credits['cast']:
    # Extract name, id, and profile_path from cast and insert into a new dataframe
    if isinstance(i, list):
        for j in i:
            try:
                data.append((j['id'], j['name'], j['gender']))
            except TypeError:
                print(f"Unexpected value: {j} in {i}")

df_cast = pd.DataFrame(data, columns=['id', 'name', 'gender']).drop_duplicates(subset=['id'])

df_credits['cast'] = df_credits['cast'].apply(lambda x: [i['id'] for i in x] if isinstance(x, list) else np.nan)
df_cast = df_cast.reset_index(drop=True)

# Define the start and end dates
start_date = pd.to_datetime('1940-01-01')
end_date = pd.to_datetime('2000-12-31')
# Create a date range
dates = pd.date_range(start_date, end_date).tolist()
# Generate a list of random birthdates for each row in the dataframe
random_birthdates = random.choices(dates, k=df_cast.shape[0])
# Convert the list to a pandas series and assign it as a new column in the dataframe
df_cast['birthdate'] = pd.Series(random_birthdates).dt.date

# Add a new column to df_cast called popularity, and get random popularity for each actor between 0 and 100
df_cast['popularity'] = np.random.randint(0, 100, df_cast.shape[0])

# Create a mapping dictionary
gender_map = {0: 'Not specified', 1: 'Female', 2: 'Male'}

# Use the map function to replace the numbers with their corresponding genders
df_cast['gender'] = df_cast['gender'].map(gender_map)

df_cast = df_cast.sort_values(by=['id'])
df_cast = df_cast.reset_index(drop=True)

In [ ]:
# rename df_credits['cast'] to df_credits['cast_id']
df_credits = df_credits.rename(columns={'cast': 'actor_id'})
df_credits.head()
# create a new dataframe with just id and cast_id
df_movie_cast = df_credits[['id', 'actor_id']]
# explode cast_id
df_movie_cast = df_movie_cast.explode('actor_id')
df_movie_cast = df_movie_cast.reset_index(drop=True)
df_movie_cast = df_movie_cast.rename(columns={'id': 'movie_id'})

In [ ]:
df_movie_cast = df_movie_cast.sort_values(by=['movie_id'])
df_movie_cast.drop_duplicates(subset=['movie_id', 'actor_id'], inplace=True)
df_movie_cast = df_movie_cast.dropna()

In [ ]:
import ast

def extract_director_names(crew_info_str):
    crew_info = ast.literal_eval(crew_info_str)
    for crew_member in crew_info:
        if crew_member['job'] == 'Director':
            return crew_member['name']
    return None

def extract_director_id(crew_info_str):
    crew_info = ast.literal_eval(crew_info_str)
    for crew_member in crew_info:
        if crew_member['job'] == 'Director':
            return crew_member['id']
    return None

df_credits['director_name'] = df_credits['crew'].apply(extract_director_names)
df_credits['director_id'] = df_credits['crew'].apply(extract_director_id)
# turn director_id from float to int
df_credits.dropna(inplace=True)
df_directors = df_credits[['id','director_id', 'director_name']]
# convert df_directors to dataframe
df_directors = pd.DataFrame(df_directors)
df_directors['director_id'] = df_directors['director_id'].astype(int)


In [ ]:
df_credits.sort_values(by=['id'], inplace=True)

In [ ]:
# join df and df_credits on id column
df = df.merge(df_directors, on='id', how='left')
df = df.dropna()
df['director_id'] = df['director_id'].astype(int)

In [ ]:
df = df.reset_index(drop=True)
# drop director_name column
df = df.drop(columns=['director_name'])

df_directors = df_directors.reset_index(drop=True)
# drop movie id from directors
df_directors = df_directors.drop(columns=['id'])

In [ ]:
# Read mock_data.csv
df_mock = pd.read_csv('MOCK_DATA.csv')
# duplicate the rows 10 times to make the dataset bigger
df_mock = pd.concat([df_mock]*80, ignore_index=True)
# drop id column
df_mock = df_mock.drop(columns=['id'])

# join on index of dataframes
df_directors =  df_directors.join(df_mock)
df_directors = df_directors.rename(columns={'director_name': 'name'})
df_directors = df_directors.reset_index(drop=True)
# Rename the column name director_id to id
df_directors = df_directors.rename(columns={'director_id': 'id'})

In [ ]:
# clean duplicates on primary key
df_directors.drop_duplicates(subset=['id'], inplace=True)
df_directors = df_directors.reset_index(drop=True)
df.drop_duplicates(subset=['id'], inplace=True)
df = df.reset_index(drop=True)

# drop duplicates in df_genre depending on id and genre
df_genres.drop_duplicates(subset=['id', 'genre'], inplace=True)
df_genres = df_genres.reset_index(drop=True)

# remove rows from movie_genre if the movie id is not in movie
df_genres = df_genres[df_genres['id'].isin(df['id'])]

In [ ]:
# Remove row in df_movie_cast if movie_id is not in df
df_movie_cast = df_movie_cast[df_movie_cast['movie_id'].isin(df['id'])]
# Remove row in df_movie_cast if actor_id is not in df_cast
df_movie_cast = df_movie_cast[df_movie_cast['actor_id'].isin(df_cast['id'])]
df_movie_cast = df_movie_cast.reset_index(drop=True)

In [ ]:
from sqlalchemy import create_engine
engine = create_engine('URL')

df_directors.to_sql('director', con=engine, if_exists='append', index=False)

In [ ]:
df.to_sql('movie', con=engine, if_exists='append', index=False)

In [ ]:
df_genres.to_sql('movie_genre', con=engine, if_exists='append', index=False)

In [ ]:
df_cast.to_sql('actor', con=engine, if_exists='append', index=False)
df_movie_cast.to_sql('movie_actor', con=engine, if_exists='append', index=False)